# LOGIN AND FETCH TOKEN

In [3]:
import globus_sdk
import requests
import json
from inference_auth_token import get_access_token

# Authenticate and collect access token
# If tokens already exist, this will reuse them and fresh the access token if necessary
access_token = get_access_token()

Please go to this URL and login:

https://auth.globus.org/v2/oauth2/authorize?client_id=58fdd3bc-e1c3-4ce5-80ea-8d6b87cfb944&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2F681c10cc-f684-4540-bcd7-0b4df3bc26ef%2Faction_all&state=_default&response_type=code&code_challenge=ebucvVdS457Amtj69rBsQ24mDj_BOCJ9TxkBCMHn-zs&code_challenge_method=S256&access_type=online

Enter the code you just received: ^C
Traceback (most recent call last):
  File "/Users/adityatanikanti/Codes/inference-endpoints/generate_auth_token.py", line 29, in <module>
    auth_code = input("Enter the code you just received: ")
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


## INFERENCE USING VLLM

### CHAT COMPLETIONS

In [5]:
model =  "meta-llama/Meta-Llama-3-8B-Instruct"
query = "List all proteins that interact with RAD51"

# URL to the inference gateway
url = "https://data-portal-dev.cels.anl.gov/resource_server/sophia/vllm/v1/chat/completions"

# Add access token to the headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
# Prepare the inference request
data = {
        "model": model,
        "messages":[{"role": "user", "content": query}],
}

# Convert data into Json
data_json = json.dumps(data)

# Send the post request to the relay server
response = requests.post(url, data=data_json, headers=headers)

# Print inference response
print(response.status_code)
print(response.json())

200
{'server_response': '{\n    "id": "cmpl-364f2af1491a4a3ebf4d670ac55867e9",\n    "object": "chat.completion",\n    "created": 1721315660,\n    "model": "meta-llama/Meta-Llama-3-8B-Instruct",\n    "choices": [\n        {\n            "index": 0,\n            "message": {\n                "role": "assistant",\n                "content": "RAD51 is a key protein involved in homologous recombination (HR) and DNA repair. It has been shown to interact with numerous proteins to regulate its activity and function. Here is a list of some of the proteins that interact with RAD51:\\n\\n1. BRCA1: RAD51 interacts with BRCA1, a tumor suppressor protein, to promote HR and DNA repair.\\n2. BRCA2: RAD51 also interacts with BRCA2, another tumor suppressor protein, to facilitate HR and DNA repair.\\n3. PALB2: PALB2, a protein involved in HR and DNA repair, interacts with RAD51 and BRCA2 to regulate their activity.\\n4. FANCD2: FANCD2,",\n                "tool_calls": []\n            },\n            "lo

### CHAT COMPLETIONS USING PYTHON OPENAI API

In [ ]:
from openai import OpenAI
 
# Set OpenAI's API key and API base to use vLLM's API server.
# localurl = "http://localhost:8000/resource_server/polaris/vllm/v1"
openai_api_key = access_token
openai_api_base = "https://data-portal-dev.cels.anl.gov/resource_server/sophia/vllm/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

model="meta-llama/Meta-Llama-3-8B-Instruct"
query = "A detailed description of the biochemical \
            function 5-(hydroxymethyl)furfural/furfural transporter is"
# sampling_params = SamplingParams({"prompt_logprobs": 1, "logprobs": 1))

chat_response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    logprobs=True,
    top_logprobs=1,
    messages=[
        {"role": "user", "content": query},
    ],
    max_tokens=2056
)
print("Chat response:", chat_response)